In [115]:
import numpy as np
import pandas as pd
import sklearn as sk
import sklearn.model_selection
import sklearn.neural_network

In [116]:
features = pd.read_csv('hb_train_feature.csv')
truths = pd.read_csv('hb_train_truth.csv')
truths.head()

,f1,f2,f3,action
0,0.868421,0.078947,0.052632,1.0
1,0.300000,0.550000,0.150000,2.0
2,0.450000,0.000000,0.550000,3.0
3,0.421053,0.157895,0.421053,1.0
4,0.125000,0.550000,0.325000,2.0


In [122]:
# add features to data

In [117]:
# add unique NE as features
!python -m pip install nashpy
import nashpy
import warnings
warnings.filterwarnings('ignore')

UNEs, NE_cnts = [], []
for i in range(features.shape[0]):
    payoffs = np.array(features.iloc[i]) 
    R = payoffs[:9].reshape(3,3)
    C = payoffs[9:18].reshape(3,3)
    rps = nashpy.Game(R, C)
    eqs = list(rps.support_enumeration()) # could be unique or multiple (PNE MNE)
    UNE = list(np.concatenate(eqs[0])) if len(eqs)==1 else list(np.zeros(6))
    NE_cnt = len(eqs)    
    UNEs.append(UNE)
    NE_cnts.append(NE_cnt)

# append to features
names = ['UNE_r1', 'UNE_r2','UNE_r3','UNE_c1','UNE_c2','UNE_c3']
for i in range(6):
    features[names[i]] = [UNE[i] for UNE in UNEs]
features['NE_cnts'] = NE_cnts

In [118]:
# splits into training and test sets
X_training, X_test = sk.model_selection.train_test_split(features)
y_training, y_test = truths[['f1', 'f2', 'f3']].loc[X_training.index], truths[['f1', 'f2', 'f3']].loc[X_test.index]

# standardizes features 
scaler = sk.preprocessing.StandardScaler()
scaler.fit_transform(X_training)
scaler.transform(X_test)
scaler.transform(X_training)
X_training.head()

,r11,r12,r13,r21,r22,r23,r31,r32,r33,c11,...,c31,c32,c33,UNE_r1,UNE_r2,UNE_r3,UNE_c1,UNE_c2,UNE_c3,NE_cnts
102,10.0,50.0,60.0,70.0,30.0,60.0,20.0,50.0,70.0,80.0,...,20.0,90.0,40.0,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1
219,30.0,40.0,90.0,60.0,30.0,20.0,80.0,10.0,90.0,40.0,...,90.0,40.0,60.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2
5,50.0,40.0,65.0,70.0,55.0,35.0,60.0,100.0,10.0,50.0,...,65.0,35.0,10.0,0.345133,0.283186,0.371681,0.345133,0.283186,0.371681,1
116,40.0,20.0,10.0,60.0,20.0,90.0,40.0,90.0,60.0,60.0,...,70.0,40.0,50.0,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1
140,50.0,60.0,90.0,60.0,20.0,10.0,50.0,60.0,40.0,30.0,...,70.0,80.0,10.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3


In [119]:
# trains separate neural net to predict each specific player's actions
classifier = sk.neural_network.MLPRegressor(solver='lbfgs', random_state=1, max_iter=1000000000, verbose=True)
classifier.fit(X_training, y_training)
classifier.score(X_test, y_test)


-21.04385613051279

In [121]:
# function to test # of top actions predicted correctly
top_actions_test = truths['action'].loc[y_test.index].reset_index(drop=True)
predicted_frequency = pd.DataFrame(classifier.predict(X_test))
predicted_actions = predicted_frequency.idxmax(axis=1).apply(lambda x: x + 1)

# number of correct top-action predictions on test data
result = (top_actions_test==predicted_actions).value_counts()
print("A =", result[1]/(result[0]+result[1]))

A = 0.49206349206349204
